# Train

> Dev Notebook

In [1]:
#| default_exp infer

In [2]:
# | hide
# | hide
%reload_ext autoreload
%reload_ext nb_black
%autoreload 2
from nbdev.showdoc import *
import sys

__root = "../"
sys.path.append(__root)

<IPython.core.display.Javascript object>

In [3]:
# | export
from torch_snippets import *
from diffusion.config import Config
from diffusion.models import *
from diffusion.dataset import *
from diffusion.train import construct_noise_schedules

<IPython.core.display.Javascript object>

In [6]:
#| export
def denoise_add_noise(x, t, pred_noise, config, z=None):
    ab_t, a_t, b_t = construct_noise_schedules(config)
    if z is None:
        z = torch.randn_like(x)
    noise = b_t.sqrt()[t] * z
    mean = (x - pred_noise * ((1 - a_t[t]) / (1 - ab_t[t]).sqrt())) / a_t[t].sqrt()
    return mean + noise

@torch.no_grad()
def sample_ddpm(nn_model, n_sample, config, save_rate=20):
    c = config
    samples = torch.randn(n_sample, 3, c.height, c.height).to(device)
    # array to keep track of generated steps for plotting
    intermediate = []
    for i in (tracker:=track2(range(c.timesteps, 0, -1), total=c.timesteps)):
        tracker.send(f'sampling timestep {i:3d}')
        # reshape time tensor
        t = torch.tensor([i / c.timesteps])[:, None, None, None].to(device)
        # sample some random noise to inject back in. For i = 1, don't add back in noise
        z = torch.randn_like(samples) if i > 1 else 0
        eps = nn_model(samples, t)    # predict noise e_(x_t,t)
        samples = denoise_add_noise(samples, i, eps, config=config, z=z)
        if i % save_rate ==0 or i==c.timesteps or i<8:
            intermediate.append(samples.detach().cpu().numpy())
    intermediate = np.stack(intermediate)
    return samples, intermediate


<IPython.core.display.Javascript object>

In [5]:
# | hide
import nbdev

nbdev.nbdev_export()

<IPython.core.display.Javascript object>